In [ ]:
###
### This cell is required only for visualisation and can be removed
### If you do decide to keep it ensure that graphviz is installed and added to PATH
###

!pip install dsplot
from dsplot.tree import BinaryTree

In [ ]:

from math import log2


In [22]:
dataLabels = ['Ear Shape','Face Shape','Whiskers','Cat']
dataDesc = [['Pointy','Floppy'],['Round','Not Round'],['Present','Absent'],['Cat','Not Cat']]
dataset = [
    [1,1,1,1],
    [0,0,1,1],
    [0,1,0,0],
    [1,0,1,0],
    [1,1,1,1],
    [1,1,0,1],
    [0,0,0,0],
    [1,1,0,1],
    [0,1,0,0],
    [0,1,0,0]
]

In [23]:
#c is number of true cases
def H(c):
    d = 1 - c
    if c == 0 or c == 1:
        return 0
    return (-c * log2(c)) + (-d * log2(d))

In [24]:
class Node:
    def __init__(self, data):
        self.data = data
        self.split = -1
        self.splitNotTrue = None
        self.splitTrue = None
        self.parent = None
        self.trueCases = len([0 for i in self.data if i[3]==1])
        self.totalCases = len(self.data)
        if self.totalCases == 0: 
            self.entropy = 0
        else:
            self.entropy = H(self.trueCases/self.totalCases)
    
    def setSplit(self, split):
        self.split = split
        if split != -1:
            isNotTrue = []
            isTrue = []
            for i in self.data:
                if i[split] == 0:
                    isNotTrue.append(i)
                else:
                    isTrue.append(i)
            self.splitNotTrue = Node(isNotTrue)
            self.splitTrue = Node(isTrue)
            self.splitNotTrue.parent = self
            self.splitTrue.parent = self
        else:
            self.splitTrue = None
            self.splitNotTrue = None
            
    def __repr__(self):
        return f"\n\nData: {self.data}\n Split At: {self.split}\n Entropy: {self.entropy}\n\n"
    
    def toString(self):
        return f"Length: {(self.data)}, Split At: {self.split}, Entropy: {round(self.entropy,3)}\t"

In [25]:
class Tree:
    def __init__(self, data):
        self.data = data
        self.tree = []
    def GenerateTree(self):
        self.tree = []
        for i in self.data:
            self.tree.append([])
            for j in i:
                if j != None:
                    if len(j.data) != 0:
                        self.tree[-1].append(j.split)
                        
        tempTree = []
        for i in self.tree:
            if i.count(-1)!=len(i):
                tempTree.append([])
                for j in i:
                    if j != None:
                        tempTree[-1].append(j)
        self.tree = tempTree
        del(tempTree)
        return self.tree
    
    def __repr__(self):
        s = ""
        for i in self.data:
            s+="\n\n"
            for j in i:
                if j != None:
                    if len(j.data) != 0:
                        s+=j.toString()
        return s

In [26]:
#number of case that are true
p = len([0 for i in dataset if i[-1]==1])/len(dataset)

rootNode = Node(dataset)

In [27]:
# s1: total cases in branch one
# s2: total cases in branch two
# t1: true cases in branch one
# t2: true cases in branch two
# maximise wh each iteration
def redH(s1,s2,t1,t2):
    wh1 = (s1/(s1+s2))*(H(t1/s1) if s1!=0 else 0)
    wh2 = (s2/(s1+s2))*(H(t2/s2) if s2!=0 else 0)
    wh = H((t1+t2)/(s1+s2))-(wh1+wh2)
    return wh

In [28]:
layers = [[rootNode]]
count = 0
while count < 5:
    bottomlayer = layers[-1]
    layers.append([])
    flag = 1
    for i in bottomlayer:
        if i != None:
            if i.entropy != 0:
                flag = 0
    if flag == 1:
        break
    for i in bottomlayer:
        if i.entropy != 0:
            s = -1
            minh = 0
            for j in range(len(dataset[0])-1):
                if i.parent != None:
                    i.setSplit(j)
                    h = redH(i.splitNotTrue.totalCases, i.splitTrue.totalCases, i.splitNotTrue.trueCases, i.splitTrue.trueCases)
                    if h > minh:
                        s = j
                        minh = h
                    i.setSplit(-1)
                else:
                    i.setSplit(j)
                    h = redH(i.splitNotTrue.totalCases, i.splitTrue.totalCases, i.splitNotTrue.trueCases, i.splitTrue.trueCases)
                    if h > minh:
                        s = j
                        minh = h
                    i.setSplit(-1)
            i.setSplit(s)
            layers[-1].append(i.splitNotTrue)
            layers[-1].append(i.splitTrue)
        else:
            layers[-1].append(i)
    count += 1
d = Tree(layers)

COMPLETED!


In [29]:
d.GenerateTree()
tree = d.tree

In [31]:
###
### This cell is only for visualisation purposes and may be removed
###

tempTree = []
for i in tree:
    for x in range(len(i)):
        tempTree.append(dataLabels[int(i[x])])
tree = tempTree

_tree = BinaryTree(nodes=tree)
_tree.plot('tree.jpg')